In [1]:
import numpy as np

In [2]:
class Operation:
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.consumers = []
        for node in input_nodes:
            node.consumers.append(self)
        _default_graph.operations.append(self)
        
    def compute(self):
        pass
        

In [3]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_value, y_value):
        return x_value + y_value
        

In [4]:
class matmul(Operation):
    def __init__(self, a, b):
        super().__init__([a,b])
        
    def compute(self, a_value, b_value):
        return a_value.dot(b_value)

In [5]:
class placeholder:
    def __init__(self):
        self.consumers = []
        _default_graph.placeholders.append(self)
        
    

In [6]:
class Variable:
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.consumers = []
        _default_graph.variables.append(self)

In [7]:
class Graph:
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def as_default(self):
        global _default_graph
        _default_graph = self

In [8]:
#Testing Time (Ax + b)

In [9]:
Graph().as_default()

#matrix: first element is first row
A = Variable([[1, 0], [0, -1]]) 
b = Variable([1, 1])
x = placeholder()

y = matmul(A, x)
z = add(y, b)

In [14]:
class Session:
    def run(self, operation, feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        print(operation)
        for node in nodes_postorder:
            print(node)
            if type(node) == placeholder:
                #placeholder values are passed in feed_dict with the placeholder as key
                node.output = feed_dict[node] 
            elif type(node) == Variable:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)         
                
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        #operation points to what the last node in nodes_postorder_ points
        return operation.output 
        
        
def traverse_postorder(operation):
    nodes_postorder = []

    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [15]:
#run
session = Session()
output = session.run(z, {
    x: [1,2]
})
print(output)

[ 2 -1]
